# XGBoost with Optuna tuning
* doc: 
https://github.com/optuna/optuna
* thanks to: 
https://www.kaggle.com/miklgr500/optuna-xgbclassifier-parameters-optimize


In [ ]:
import xgboost as xgb
import numpy as np
import pandas as pd
import optuna
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

In [ ]:
train = pd.read_csv("../input/predicting-pulsar-starintermediate/pulsar_data_train.csv")
test = pd.read_csv("../input/predicting-pulsar-starintermediate/pulsar_data_test.csv")

In [ ]:
train.head()

In [ ]:
test.head()

In [ ]:
train['target_class'].value_counts()

In [ ]:
train=train.fillna(0)
test=test.fillna(0)

In [ ]:
target = train['target_class']
data = train.drop('target_class',axis=1)
test = test.drop('target_class',axis=1)

In [ ]:
columns=data.columns.to_list()
columns

In [ ]:
def objective(trial,data=data,target=target):
    
    train_x, test_x, train_y, test_y = train_test_split(data, target, test_size=0.2,random_state=42)
    param = {
        'tree_method':'gpu_hist',  # using the GPU
        'lambda': trial.suggest_loguniform('lambda',1e-3,10.0),
        'alpha': trial.suggest_loguniform('alpha',1e-3,10.0),
        'colsample_bytree': trial.suggest_categorical('colsample_bytree', [0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0]),
        'subsample': trial.suggest_categorical('subsample', [0.4,0.5,0.6,0.7,0.8,1.0]),
        'learning_rate': trial.suggest_categorical('learning_rate', [0.008,0.01,0.012,0.014,0.016,0.018,0.02]),
        'n_estimators': 4000,
        'max_depth': trial.suggest_categorical('max_depth', [5,7,9,11,13,15,17,20]),
        'random_state': trial.suggest_categorical('random_state', [24,48,2020]),
        'min_child_weight': trial.suggest_int('min_child_weight', 1,300),
    }
    model = xgb.XGBRegressor(**param)      
    model.fit(train_x,train_y,eval_set=[(test_x,test_y)],early_stopping_rounds=100,verbose=False)
    preds = model.predict(test_x)
    rmse = mean_squared_error(test_y, preds,squared=False)
    
    return rmse

In [ ]:
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=20)
print('Number of finished trials:', len(study.trials))
print('Best trial:', study.best_trial.params)

In [ ]:
study.trials_dataframe()

In [ ]:
# shows the scores from all trials
optuna.visualization.plot_optimization_history(study)

In [ ]:
# interactively visualizes the hyperparameters and scores
optuna.visualization.plot_parallel_coordinate(study)

In [ ]:
# shows the evolution of the search
optuna.visualization.plot_slice(study)

In [ ]:
# parameter interactions on an interactive chart.
optuna.visualization.plot_contour(study, params=['lambda','learning_rate'])

In [ ]:
# Visualize parameter importances.
optuna.visualization.plot_param_importances(study)

In [ ]:
# Visualize empirical distribution function
optuna.visualization.plot_edf(study)

In [ ]:
Best_trial= {'lambda': 1.9029439063696163, 'alpha': 0.028064106937089998, 
             'colsample_bytree': 0.7, 'subsample': 0.4, 'learning_rate': 0.018, 
             'max_depth': 5, 'random_state': 48, 'min_child_weight': 3}

In [ ]:
preds = np.zeros(test.shape[0])
kf = KFold(n_splits=5,random_state=48,shuffle=True)

for trn_idx, test_idx in kf.split(train[columns],train['target_class']):
    X_tr,X_val=train[columns].iloc[trn_idx],train[columns].iloc[test_idx]
    y_tr,y_val=train['target_class'].iloc[trn_idx],train['target_class'].iloc[test_idx]
    model = xgb.XGBRegressor(**Best_trial)
    model.fit(X_tr,y_tr,eval_set=[(X_val,y_val)],early_stopping_rounds=100,verbose=False)
    preds+=model.predict(test[columns])/kf.n_splits
    rmse=mean_squared_error(y_val, model.predict(X_val),squared=False)
    print(rmse)

In [ ]:
preds2=np.where(preds<0.5,0,1)
preds2[0:10]